## This Jupyter file is to prepare the data for NLP processing of Review text for recommending improvement points for low rated restaurants.

In [1]:
#Connect to MongoDB
import pymongo
from pymongo import MongoClient

import pandas as pd
from pandas import DataFrame

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
    
client = MongoClient('mongodb://localhost:27017')
#Get the yelpDB database
db = client.yelp

### Read AZ_restaurant_review_data collection from mongodb.   
#### This collection contains the Reviews details of each business. Filter the content for top 5 cities (Based on the EDA)

In [2]:
AZ_restaurant_review_data = db.AZ_restaurant_review_data.find({'city':{"$in" : ['Phoenix','Scottsdale','Mesa','Tempe','Chandler']}})
AZ_restaurant_review_data_df = DataFrame(list(AZ_restaurant_review_data))
AZ_restaurant_review_data_df.shape

(5451, 16)

### Read the cleaned AZ_Restaurant_Final_Clean_Data.csv file with flattened restaurant details. This is the output file from EDA & data cleansing. 

In [3]:
AZ_Restaurant_Flat_Data= pd.read_csv("AZ_Restaurant_Final_Clean_Data.csv", sep =";", index_col="Unnamed: 0")
AZ_Restaurant_Flat_Data.shape

(7367, 234)

In [4]:
#Merge the above 2 dataframes to a single dataframe AZ_restrnt_flatten_df
AZ_restrnt_flatten_df = pd.merge(AZ_restaurant_review_data_df,AZ_Restaurant_Flat_Data,on = 'business_id')

#Below columns would be retained based on what goes into the UI. These columns are chosen based on the valid data available
# (least number of Nans)
column_list = ["business_id", "name_x", "address_x", "city_x", "state", "postal_code_x", "review_count_x", "review_details",
               "stars_x","RestaurantsPriceRange2"]
AZ_restrnt_flatten_df = AZ_restrnt_flatten_df[column_list]

# Rename the column header with new name
column_list_updated = ["Business_id", "Name", "Address", "City", "State", "Postal_code", "Review_count", "Review_details",
                       "Restaurant_ratings","RestaurantsPriceRange2"]
AZ_restrnt_flatten_df.columns = column_list_updated
print(AZ_restrnt_flatten_df.shape)
AZ_restrnt_flatten_df.head(3)

(5451, 10)


,Business_id,Name,Address,City,State,Postal_code,Review_count,Review_details,Restaurant_ratings,RestaurantsPriceRange2
0,rDMptJYWtnMhpQu_rRXHng,McDonald's,719 E Thunderbird Rd,Phoenix,AZ,85022,10,"[{'review_id': 'bABGON0ehmb7MBJrI02l7Q', 'user...",1.0,1.0
1,1WBkAuQg81kokZIPMpn9Zg,Charr An American Burger Bar,"777 E Thunderbird Rd, Ste 107",Phoenix,AZ,85022,232,"[{'review_id': 'xj31weGCI08_2xGQwEx0hA', 'user...",3.0,2.0
2,iPa__LOhse-hobC2Xmp-Kw,McDonald's,1635 E Camelback Rd,Phoenix,AZ,85016,34,"[{'review_id': 'myIo7pMWP6B4XiZYv2EsSQ', 'user...",3.0,1.0


**Create function to flatten the user review details for each restaurant**

In [5]:
def review_flatten(Review_data_Price_rate):
    i = 0
    j = 0
    Price_review_df = pd.DataFrame()
    while i < len(Review_data_Price_rate):

        review1 = pd.DataFrame(Review_data_Price_rate.iloc[i]['Review_details'])
        review1['Business_id'] = Review_data_Price_rate.iloc[i]['Business_id']
        Price_review_df = pd.concat([review1,Price_review_df])
             
        #percentage = (i/len(Review_data_Price_rate))*100
        #if(int(round(percentage)) % 10 == 0):
        #    print (int(round(percentage)) ,"%", i,end =' ')
        i +=1
    return Price_review_df

In [6]:
AZ_restrnt_review_flatten = pd.DataFrame()
AZ_restrnt_review_flatten = review_flatten(AZ_restrnt_flatten_df)

AZ_restrnt_review_flatten.rename(columns={'stars':'Review_ratings', 'user_id' : 'User_id','text':'Review_text'}, inplace=True)
AZ_restrnt_review_flatten.head(3)

C:\Users\Joy\Anaconda_Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,Business_id,cool,date,funny,review_id,Review_ratings,Review_text,useful,User_id
0,UdEmYOnk2iJDY9lpEPAlJQ,2.0,2014-12-30,0.0,OwX02bSKJ0dZVUhlHcpNiw,4.0,"Solid, cheap, delicious Italian. Good calzone...",2.0,mvq6mTZBd3mdDg_bZRWiuQ
1,UdEmYOnk2iJDY9lpEPAlJQ,2.0,2014-11-02,0.0,Ws-n-35XsXE-Cvgd8Biiig,5.0,We come here a lot. My kids love their pizza a...,2.0,eNf0HUg_VNjKe9enwimRsg
2,UdEmYOnk2iJDY9lpEPAlJQ,1.0,2015-08-29,0.0,g2cHc3gWnf0dEaWuu3RM1w,5.0,I've been coming here since I was 14 years old...,0.0,314dp8E50a7mjjSeZhKB-w


In [7]:
AZ_restrnt_review_flatten.shape

(518467, 9)

**Merge the Business and user review dataframes and remove the unwanted columns(like cool, date, funny, useful, review_id)**

In [8]:
AZ_restrnt_user_flatten_df = pd.merge(AZ_restrnt_flatten_df,AZ_restrnt_review_flatten,on = 'Business_id')
AZ_restrnt_user_flatten_df.head(2)


column_list = ["User_id","Business_id", "Name", "Address", "City", "State", "Postal_code", "Review_count",
               "Restaurant_ratings","RestaurantsPriceRange2","Review_ratings","Review_text"]
AZ_restrnt_user_flatten_df = AZ_restrnt_user_flatten_df[column_list]
AZ_restrnt_user_flatten_df.head(2)

,User_id,Business_id,Name,Address,City,State,Postal_code,Review_count,Restaurant_ratings,RestaurantsPriceRange2,Review_ratings,Review_text
0,Ck73f1qtZbu68F_vjzsBrQ,rDMptJYWtnMhpQu_rRXHng,McDonald's,719 E Thunderbird Rd,Phoenix,AZ,85022,10,1.0,1.0,1.0,The speed of delivery of my food order was ter...
1,F95NFEFwuwA__SIRt9IJNA,rDMptJYWtnMhpQu_rRXHng,McDonald's,719 E Thunderbird Rd,Phoenix,AZ,85022,10,1.0,1.0,1.0,I stopped by for a double quarter pounder with...


In [9]:
AZ_restrnt_user_flatten_df.RestaurantsPriceRange2.value_counts()

2.0    331149
1.0    160388
3.0     20841
0.0      3400
4.0      2689
Name: RestaurantsPriceRange2, dtype: int64

In [10]:
AZ_restrnt_user_flatten_df.City.value_counts()

Phoenix       232597
Scottsdale    121869
Tempe          63114
Mesa           52812
Chandler       48075
Name: City, dtype: int64

In [11]:
AZ_restrnt_user_flatten_df = AZ_restrnt_user_flatten_df[AZ_restrnt_user_flatten_df['RestaurantsPriceRange2'] != 0.0]

In [12]:
AZ_restrnt_user_flatten_price1_df = AZ_restrnt_user_flatten_df[AZ_restrnt_user_flatten_df['RestaurantsPriceRange2'] == 1.0]
AZ_restrnt_user_flatten_price2_df = AZ_restrnt_user_flatten_df[AZ_restrnt_user_flatten_df['RestaurantsPriceRange2'] == 2.0]
AZ_restrnt_user_flatten_price3_df = AZ_restrnt_user_flatten_df[AZ_restrnt_user_flatten_df['RestaurantsPriceRange2'] == 3.0]
AZ_restrnt_user_flatten_price4_df = AZ_restrnt_user_flatten_df[AZ_restrnt_user_flatten_df['RestaurantsPriceRange2'] == 4.0]

## Create new dataframes by Price range for low and high rated reviews.

In [13]:
Price1_lowrated_review_final = AZ_restrnt_user_flatten_price1_df[(AZ_restrnt_user_flatten_price1_df
                                ['Restaurant_ratings'] < 3) & (AZ_restrnt_user_flatten_price1_df['Review_ratings'] < 3)]
Price1_lowrated_review_final.shape

(10491, 12)

In [14]:
Price1_highrated_review_final = AZ_restrnt_user_flatten_price1_df[(AZ_restrnt_user_flatten_price1_df
                                ['Restaurant_ratings'] > 3.5) & (AZ_restrnt_user_flatten_price1_df['Review_ratings'] > 3.5)]
Price1_highrated_review_final.shape

(79725, 12)

In [15]:
Price2_lowrated_review_final = AZ_restrnt_user_flatten_price2_df[(AZ_restrnt_user_flatten_price2_df
                                ['Restaurant_ratings'] < 3) & (AZ_restrnt_user_flatten_price2_df['Review_ratings'] < 3)]
Price2_lowrated_review_final.shape

(6184, 12)

In [16]:
Price2_highrated_review_final = AZ_restrnt_user_flatten_price2_df[(AZ_restrnt_user_flatten_price2_df
                                ['Restaurant_ratings'] > 3.5) & (AZ_restrnt_user_flatten_price2_df['Review_ratings'] > 3.5)]
Price2_highrated_review_final.shape

(165010, 12)

In [17]:
Price3_lowrated_review_final = AZ_restrnt_user_flatten_price3_df[(AZ_restrnt_user_flatten_price3_df
                                ['Restaurant_ratings'] < 3) & (AZ_restrnt_user_flatten_price3_df['Review_ratings'] < 3)]
Price3_lowrated_review_final.shape

(67, 12)

In [18]:
Price3_highrated_review_final = AZ_restrnt_user_flatten_price3_df[(AZ_restrnt_user_flatten_price3_df
                                ['Restaurant_ratings'] > 3.5) & (AZ_restrnt_user_flatten_price3_df['Review_ratings'] > 3.5)]
Price3_highrated_review_final.shape

(13169, 12)

In [19]:
Price4_lowrated_review_final = AZ_restrnt_user_flatten_price4_df[(AZ_restrnt_user_flatten_price4_df
                                ['Restaurant_ratings'] < 3) & (AZ_restrnt_user_flatten_price4_df['Review_ratings'] < 3)]
Price4_lowrated_review_final.shape

(25, 12)

In [20]:
Price4_highrated_review_final = AZ_restrnt_user_flatten_price4_df[(AZ_restrnt_user_flatten_price4_df
                                ['Restaurant_ratings'] > 3.5) & (AZ_restrnt_user_flatten_price4_df['Review_ratings'] > 3.5)]
Price4_highrated_review_final.shape

(1650, 12)

In [21]:
#Concat the 8 dataframes into a single dataframe
AZ_restaurant_low_high_review_final = pd.concat([Price1_lowrated_review_final,Price1_highrated_review_final,
                                                 Price2_lowrated_review_final,Price2_highrated_review_final,
                                                 Price3_lowrated_review_final,Price3_highrated_review_final,
                                                 Price4_lowrated_review_final,Price4_highrated_review_final,])

In [22]:
print(AZ_restaurant_low_high_review_final.shape)
AZ_restaurant_low_high_review_final.head(3)

(276321, 12)


,User_id,Business_id,Name,Address,City,State,Postal_code,Review_count,Restaurant_ratings,RestaurantsPriceRange2,Review_ratings,Review_text
0,Ck73f1qtZbu68F_vjzsBrQ,rDMptJYWtnMhpQu_rRXHng,McDonald's,719 E Thunderbird Rd,Phoenix,AZ,85022,10,1.0,1.0,1.0,The speed of delivery of my food order was ter...
1,F95NFEFwuwA__SIRt9IJNA,rDMptJYWtnMhpQu_rRXHng,McDonald's,719 E Thunderbird Rd,Phoenix,AZ,85022,10,1.0,1.0,1.0,I stopped by for a double quarter pounder with...
2,2gWCW1oEuyhaxrlTTghvtQ,rDMptJYWtnMhpQu_rRXHng,McDonald's,719 E Thunderbird Rd,Phoenix,AZ,85022,10,1.0,1.0,1.0,I was told tonight at 8:30 pm that they were n...


**Export the dataframe to be persisted for further processing**

In [23]:
AZ_restaurant_low_high_review_final.to_csv("AZ_restaurant_low_high_review_final.csv", index=False, sep='\t', encoding='utf-8')

In [13]:
import pandas as pd
AZ_restaurant_low_high_review_final1 = pd.read_csv("AZ_restaurant_low_high_review_final.csv",sep='\t')
AZ_restaurant_low_high_review_final1.shape

(276327, 12)

In [14]:
AZ_restaurant_low_high_review_final1.RestaurantsPriceRange2.value_counts()

2.0    171194
1.0     90216
3.0     13236
4.0      1675
Name: RestaurantsPriceRange2, dtype: int64